**Using EEG Models from ARL on our dataset**

This notebook contains EEG Models from the Army Research Laboratory. Experiments were ran using these models on our single trial files dataset to see if they would get better accuracy. These models were also used on the MNE dataset to see if they could achieve better accuracy on that dataset than our model did. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install mne

**Load our Datset**

In [ ]:
%cd drive/My \Drive/
%run LoadData.ipynb

**Imports**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling2D, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv1D, SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import confusion_matrix
import mne
from mne.datasets import sample
import numpy as np
from mne import io, pick_types, read_events, Epochs, EvokedArray

**Load MNE dataset**

In [ ]:
data_path = sample.data_path()

raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
event_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
tmin, tmax = -0.1, 0.3
event_id = {'Auditory/Left': 1, 'Auditory/Right': 2,
            'Visual/Left': 3, 'Visual/Right': 4}
n_filter = 3

Using default location ~/mne_data for sample...
Creating ~/mne_data



Verifying hash 12b75d1cb7df9dfb4ad73ed82f61094f.
Decompressing the archive: /root/mne_data/MNE-sample-data-processed.tar.gz
(please be patient, this can take some time)
Successfully extracted to: ['/root/mne_data/MNE-sample-data']
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


In [ ]:
# Setup for reading the raw data
raw = io.read_raw_fif(raw_fname, preload=True)
raw.filter(1, 20, fir_design='firwin')
events = read_events(event_fname)

#get eeg channels
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

epochs = Epochs(raw, events, event_id, tmin, tmax, proj=False,
                picks=picks, baseline=None, preload=True,
                verbose=False)

# Get the labels and decrement them (to work for confusion matrix / machine learning)
labels = epochs.events[:, -1] - 1

Opening raw data file /root/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0
Reading 0 ... 41699  =      0.000 ...   277.709 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 497 samples (3.310 sec)



**Convert data to 3d array and transpose it**

In [ ]:
#array3d = epochs.get_data(picks='eeg')
array3d = epochs.get_data(picks = epochs.ch_names)
#transpose array
array = array3d.swapaxes(1,2)
array.shape

(288, 61, 59)

***EEGNet Model***

In [ ]:
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

**Old EEGNet Model**

In [ ]:
def EEGNet_old(nb_classes, Chans = 64, Samples = 128, regRate = 0.0001,
           dropoutRate = 0.25, kernels = [(2, 32), (8, 4)], strides = (2, 4)):
    """ Keras Implementation of EEGNet_v1 (https://arxiv.org/abs/1611.08024v2)
    This model is the original EEGNet model proposed on arxiv
            https://arxiv.org/abs/1611.08024v2
    
    with a few modifications: we use striding instead of max-pooling as this 
    helped slightly in classification performance while also providing a 
    computational speed-up. 
    
    Note that we no longer recommend the use of this architecture, as the new
    version of EEGNet performs much better overall and has nicer properties.
    
    Inputs:
        
        nb_classes     : total number of final categories
        Chans, Samples : number of EEG channels and samples, respectively
        regRate        : regularization rate for L1 and L2 regularizations
        dropoutRate    : dropout fraction
        kernels        : the 2nd and 3rd layer kernel dimensions (default is 
                         the [2, 32] x [8, 4] configuration)
        strides        : the stride size (note that this replaces the max-pool
                         used in the original paper)
    
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    layer1       = Conv2D(16, (Chans, 1), input_shape=(Chans, Samples, 1),
                                 kernel_regularizer = l1_l2(l1=regRate, l2=regRate))(input_main)
    layer1       = BatchNormalization(axis=1)(layer1)
    layer1       = Activation('elu')(layer1)
    layer1       = Dropout(dropoutRate)(layer1)
    
    permute_dims = 2, 1, 3
    permute1     = Permute(permute_dims)(layer1)
    
    layer2       = Conv2D(4, kernels[0], padding = 'same', 
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(permute1)
    layer2       = BatchNormalization(axis=1)(layer2)
    layer2       = Activation('elu')(layer2)
    layer2       = Dropout(dropoutRate)(layer2)
    
    layer3       = Conv2D(4, kernels[1], padding = 'same',
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(layer2)
    layer3       = BatchNormalization(axis=1)(layer3)
    layer3       = Activation('elu')(layer3)
    layer3       = Dropout(dropoutRate)(layer3)
    
    flatten      = Flatten(name = 'flatten')(layer3)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

**SSVEP Model**

In [ ]:
def EEGNet_SSVEP(nb_classes = 12, Chans = 8, Samples = 256, 
             dropoutRate = 0.5, kernLength = 256, F1 = 96, 
             D = 1, F2 = 96, dropoutType = 'Dropout'):
    """ SSVEP Variant of EEGNet, as used in [1]. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. 
      D               : number of spatial filters to learn within each temporal
                        convolution.
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
      
      
    [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6). 
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

***Shallow ConvNet Model***

In [ ]:
# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))  

In [ ]:
def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 25), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 75), strides=(1, 15))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

***Deep ConvNet Model***

In [ ]:
def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):
    """ Keras implementation of the Deep Convolutional Network as described in
    Schirrmeister et. al. (2017), Human Brain Mapping.
    
    This implementation assumes the input is a 2-second EEG signal sampled at 
    128Hz, as opposed to signals sampled at 250Hz as described in the original
    paper. We also perform temporal convolutions of length (1, 5) as opposed
    to (1, 10) due to this sampling rate difference. 
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                      ours        original paper
    pool_size        1, 2        1, 3
    strides          1, 2        1, 3
    conv filters     1, 5        1, 10
    
    Note that this implementation has not been verified by the original 
    authors. 
    
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 10), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 3))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 10),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 10),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 10),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

**Experiments**

In [ ]:
#transpose our trials array to make it compatible with the models
static_transpose = static_trials.swapaxes(1,2)

#static_transpose = np.empty(shape=(len(static_trials), 64, 1101))
#for x in range(0, len(static_trials)):
  #static_transpose[x] = static_trials[x].transpose()

#static_transpose.shape
#static_trials[2].transpose() == static_transpose[2] #check for equality

**K fold experiments**

**Shallow ConvNet Model**

In [ ]:
#static files
n_epochs = 25
n_splits = 4
batch_size = 64
verbose = 2

class_names = [0, 1]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D
static_4D_trials = np.expand_dims(static_transpose, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = static_labels[train_index], static_labels[test_index]

  #build model
  model = ShallowConvNet(2, Chans=64, Samples=1101, dropoutRate=0.2) 
  #compile model
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = (y_pred_prob[:,0] > 0.5) + 0
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

#print graphs to see best parameter values
#plot_metrics(history, 'loss')
#plot_metrics(history, 'accuracy')

Epoch 1/25
34/34 - 128s - loss: 0.8520 - accuracy: 0.5021 - val_loss: nan - val_accuracy: 0.5410
Epoch 2/25
34/34 - 124s - loss: 0.7260 - accuracy: 0.5035 - val_loss: 1.1331 - val_accuracy: 0.5257
Epoch 3/25
34/34 - 122s - loss: 0.7153 - accuracy: 0.5039 - val_loss: 0.9608 - val_accuracy: 0.4743
Epoch 4/25
34/34 - 123s - loss: 0.7092 - accuracy: 0.4919 - val_loss: 0.7320 - val_accuracy: 0.4826
Epoch 5/25
34/34 - 122s - loss: 0.7045 - accuracy: 0.4998 - val_loss: 0.8782 - val_accuracy: 0.5257
Epoch 6/25
34/34 - 121s - loss: 0.7028 - accuracy: 0.4873 - val_loss: 1.4831 - val_accuracy: 0.5257
Epoch 7/25
34/34 - 121s - loss: 0.7012 - accuracy: 0.5035 - val_loss: 1.2608 - val_accuracy: 0.5257
Epoch 8/25
34/34 - 129s - loss: 0.6997 - accuracy: 0.4845 - val_loss: 1.0289 - val_accuracy: 0.5257
Epoch 9/25
34/34 - 126s - loss: 0.6987 - accuracy: 0.4910 - val_loss: 1.3303 - val_accuracy: 0.5257
Epoch 10/25
34/34 - 122s - loss: 0.6983 - accuracy: 0.4826 - val_loss: 1.0367 - val_accuracy: 0.4743
Ep

KeyboardInterrupt: ignored

**EEGNet Model**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(static_transpose, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = static_labels[train_index], static_labels[test_index]

  #build model
  model = EEGNet(2, Chans=64, Samples=1101) 
  #compile model
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = (y_pred_prob[:,0] > 0.5) + 0
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

#print graphs to see best parameter values
#plot_metrics(history, 'loss')
#plot_metrics(history, 'accuracy')

Epoch 1/25
68/68 - 146s - loss: 0.6988 - accuracy: 0.5127 - val_loss: 7.6663 - val_accuracy: 0.4576
Epoch 2/25
68/68 - 146s - loss: 0.6935 - accuracy: 0.4933 - val_loss: 7.6663 - val_accuracy: 0.5049
Epoch 3/25
68/68 - 150s - loss: 0.6933 - accuracy: 0.5063 - val_loss: 7.6663 - val_accuracy: 0.5229
Epoch 4/25
68/68 - 145s - loss: 0.6932 - accuracy: 0.5132 - val_loss: 7.6663 - val_accuracy: 0.4951
Epoch 5/25
68/68 - 146s - loss: 0.6932 - accuracy: 0.4882 - val_loss: nan - val_accuracy: 0.4993
Epoch 6/25
68/68 - 145s - loss: 0.6932 - accuracy: 0.4882 - val_loss: 7.6663 - val_accuracy: 0.5049
Epoch 7/25
68/68 - 146s - loss: 0.6932 - accuracy: 0.4873 - val_loss: 7.6663 - val_accuracy: 0.4854
Epoch 8/25
68/68 - 150s - loss: 0.6932 - accuracy: 0.4822 - val_loss: 7.6663 - val_accuracy: 0.5299
Epoch 9/25
68/68 - 148s - loss: 0.6932 - accuracy: 0.4849 - val_loss: 7.6663 - val_accuracy: 0.4673
Epoch 10/25
68/68 - 147s - loss: 0.6932 - accuracy: 0.4849 - val_loss: 7.4758 - val_accuracy: 0.5076
Ep

**Old EEGNet Model**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(static_transpose, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = static_labels[train_index], static_labels[test_index]

  #build model
  model = EEGNet_old(2, Chans=64, Samples=1101) 
  #compile model
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = (y_pred_prob[:,0] > 0.5) + 0
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

#print graphs to see best parameter values
#plot_metrics(history, 'loss')
#plot_metrics(history, 'accuracy')

**SSVEP Model**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(static_transpose, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = static_labels[train_index], static_labels[test_index]

  #build model
  model = EEGNet_SSVEP(nb_classes=2, Chans=64, Samples=1101, dropoutRate=0.2, kernLength=7, F1=32, F2=32) 
  #compile model
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = (y_pred_prob[:,0] > 0.5) + 0
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

#print graphs to see best parameter values
#plot_metrics(history, 'loss')
#plot_metrics(history, 'accuracy')

**Deep ConvNet Model**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(static_transpose, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = static_labels[train_index], static_labels[test_index]

  #build model
  model = DeepConvNet(2, Chans=64, Samples=1101, dropoutRate=0.2) 
  #compile model
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = (y_pred_prob[:,0] > 0.5) + 0
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

#print graphs to see best parameter values
#plot_metrics(history, 'loss')
#plot_metrics(history, 'accuracy')

Epoch 1/25
68/68 - 89s - loss: 0.7792 - accuracy: 0.4988 - val_loss: 0.6940 - val_accuracy: 0.4937
Epoch 2/25
68/68 - 92s - loss: 0.6987 - accuracy: 0.5063 - val_loss: 0.6933 - val_accuracy: 0.4757
Epoch 3/25
68/68 - 88s - loss: 0.6972 - accuracy: 0.4826 - val_loss: 0.6934 - val_accuracy: 0.5090
Epoch 4/25
68/68 - 88s - loss: 0.6961 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.4854
Epoch 5/25
68/68 - 90s - loss: 0.6950 - accuracy: 0.5025 - val_loss: 0.6944 - val_accuracy: 0.4757
Epoch 6/25
68/68 - 88s - loss: 0.6946 - accuracy: 0.4998 - val_loss: 0.6933 - val_accuracy: 0.4896
Epoch 7/25
68/68 - 88s - loss: 0.6943 - accuracy: 0.4849 - val_loss: 0.6934 - val_accuracy: 0.4631
Epoch 8/25
68/68 - 90s - loss: 0.6941 - accuracy: 0.5002 - val_loss: 0.6932 - val_accuracy: 0.5327
Epoch 9/25
68/68 - 90s - loss: 0.6938 - accuracy: 0.4998 - val_loss: 0.6933 - val_accuracy: 0.4993
Epoch 10/25
68/68 - 90s - loss: 0.6937 - accuracy: 0.4947 - val_loss: 0.6932 - val_accuracy: 0.4715
Epoch 11/

**Old EEGNet Model with MNE dataset**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1, 2, 3]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(array, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = labels[train_index], labels[test_index]

  #build model
  model = EEGNet_old(nb_classes=4, Chans=61, Samples=59) 
  #compile model
  model.compile(optimizer=keras.optimizers.Adadelta(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

Epoch 1/25
7/7 - 0s - loss: 1.3910 - sparse_categorical_accuracy: 0.2639 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 2/25
7/7 - 0s - loss: 1.3917 - sparse_categorical_accuracy: 0.2546 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 3/25
7/7 - 0s - loss: 1.3926 - sparse_categorical_accuracy: 0.2269 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 4/25
7/7 - 0s - loss: 1.3897 - sparse_categorical_accuracy: 0.2639 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 5/25
7/7 - 0s - loss: 1.3918 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 6/25
7/7 - 0s - loss: 1.3881 - sparse_categorical_accuracy: 0.2454 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 7/25
7/7 - 0s - loss: 1.3843 - sparse_categorical_accuracy: 0.2778 - val_loss: 1.3913 - val_sparse_categorical_accuracy: 0.1944
Epoch 8/25
7/7 - 0s - loss: 1.3887 - sparse_categorical_accura

NameError: ignored

**SSVEP Model with MNE**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1, 2, 3]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(array, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = labels[train_index], labels[test_index]

  #build model
  model = EEGNet_SSVEP(nb_classes=4, Chans=59, Samples=61, dropoutRate=0.2, kernLength=5) 
  #compile model
  model.compile(optimizer=keras.optimizers.Adadelta(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

**Shallow ConvNet with MNE**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1, 2, 3]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(array, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = labels[train_index], labels[test_index]

  #build model
  model = ShallowConvNet(nb_classes=4, Chans=59, Samples=61, dropoutRate=0.2) 
  #compile model
  model.compile(optimizer=keras.optimizers.Adadelta(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')

**Deep ConvNet Model**

In [ ]:
#static files 
n_epochs = 25
n_splits = 4
batch_size = 32
verbose = 2

class_names = [0, 1, 2, 3]

accuracies = []
losses = []

predicts = []
trues = []

split_no = 1
splits = []

#make static trials 4D so it is compatible with Conv2D (trials, rows, channels, 1)
static_4D_trials = np.expand_dims(array, axis=3)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=None)

for train_index, test_index in kfold.split(static_4D_trials):
  #split data into X and y training and test sets
  static_X_train, static_X_test = static_4D_trials[train_index], static_4D_trials[test_index]
  static_y_train, static_y_test = labels[train_index], labels[test_index]

  #build model
  model = DeepConvNet(nb_classes=4, Chans=59, Samples=61, dropoutRate=0.2)
  #compile model
  model.compile(optimizer=keras.optimizers.Adadelta(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
  #fit model
  history = model.fit(static_X_train, static_y_train, epochs=n_epochs, verbose=verbose,
                    batch_size=batch_size, validation_data=(static_X_test, static_y_test))
  
  #add extra print for spacing
  print ("\n")
  #predict labels for confusion matrix
  y_true = static_y_test
  y_pred_prob = model.predict(static_X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  #add true and predicted labels to arrays
  trues.extend(y_true)
  predicts.extend(y_pred)

  #evaluate model
  loss, accuracy = model.evaluate(static_X_test, static_y_test, batch_size=batch_size, verbose=verbose)
  accuracies.append(accuracy * 100)
  losses.append(loss)
  splits.append("%s%d: %s: %.2f%%" % ("split", split_no, "ACCURACY ", accuracy*100))
  split_no += 1

#print final results
for x in range(0, n_splits):
  print(splits[x])
print("ACCURACIES: %.2f%% accuracy (+/- %.2f%%)" % (np.mean(accuracies), np.std(accuracies)))
print("LOSSES: ", np.mean(losses), " (+/-", np.std(losses), ")")

#Plot confusion matrix
conf_mat = confusion_matrix(trues, predicts, labels=class_names)
plot_confusion_matrix(cm=conf_mat, classes=class_names, title='Confusion Matrix')